<a href="https://colab.research.google.com/github/snehaa-13/shock-propagation/blob/main/temporal_shock_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

temporal shock injection

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving cleaned_bank_data.csv to cleaned_bank_data.csv
Saving cleaned_loan_data.csv to cleaned_loan_data.csv


In [ ]:
import pandas as pd

bank_df = pd.read_csv('cleaned_bank_data.csv')
loan_df = pd.read_csv('cleaned_loan_data.csv')


In [ ]:
import numpy as np
import pandas as pd

np.random.seed(42)

banks_per_step = 3
time_steps_to_shock = sorted(loan_df['time'].unique())[:5]

all_shocked_banks = set()

for t in time_steps_to_shock:
    possible_banks = list(set(loan_df[loan_df['time'] == t]['source']))
    shocked_banks = np.random.choice(possible_banks, size=min(banks_per_step, len(possible_banks)), replace=False)
    all_shocked_banks.update(shocked_banks)

    loan_df.loc[(loan_df['time'] == t) & (loan_df['source'].isin(shocked_banks)), 'shock_flag'] = 1

loan_df['shock_flag'] = loan_df['shock_flag'].fillna(0).astype(int)

loan_df[loan_df['shock_flag'] == 1].head()


,time,source,target,exposure,interest_rate,maturity,shock_flag
105,0,B11,B8,5.56,0.0429,14,1
106,0,B11,B10,14.28,0.0284,180,1
107,0,B11,B13,23.49,0.0391,180,1
108,0,B11,B28,36.89,0.0439,7,1
109,0,B11,B29,6.56,0.0315,30,1


In [ ]:
loan_df.to_csv("shocked_loan_dataset.csv", index=False)
from google.colab import files
files.download("shocked_loan_dataset.csv")


In [ ]:
import networkx as nx

graphs_by_time = {}
for t in sorted(loan_df['time'].unique()):
    G_t = nx.DiGraph()
    timestep_data = loan_df[loan_df['time'] == t]
    for _, row in timestep_data.iterrows():
        G_t.add_edge(row['source'], row['target'], weight=row['exposure'])
    graphs_by_time[t] = G_t

def compute_debtrank(G, shocked_nodes, damping_factor=0.9, max_iter=10):
    distress = {node: (1.0 if node in shocked_nodes else 0.0) for node in G.nodes()}
    state = {node: ('D' if node in shocked_nodes else 'U') for node in G.nodes()}

    for _ in range(max_iter):
        updates = False
        new_distress = distress.copy()
        for j in G.nodes():
            if state[j] == 'U':
                impact = sum(distress[i] * G[i][j].get('weight', 0) for i in G.predecessors(j) if state[i] == 'D')
                if impact > 0:
                    new_distress[j] = 1 - (1 - distress[j]) * (1 - damping_factor * impact)
                    state[j] = 'D'
                    updates = True

        for k in G.nodes():
            if state[k] == 'D':
                state[k] = 'I'
        if not updates:
            break
        distress = new_distress
    return distress

debtrank_results = []

for t in sorted(graphs_by_time.keys())[:5]:
    G_t = graphs_by_time[t]
    shocked_banks = list(loan_df[(loan_df['time'] == t) & (loan_df['shock_flag'] == 1)]['source'].unique())
    distress_dict = compute_debtrank(G_t, shocked_banks)
    for node, rank in distress_dict.items():
        debtrank_results.append({'time': t, 'bank': node, 'debt_rank': rank})

debtrank_df = pd.DataFrame(debtrank_results)
debtrank_df.head()


,time,bank,debt_rank
0,0,B0,0.0
1,0,B4,0.0
2,0,B7,0.0
3,0,B18,0.0
4,0,B20,0.0


In [ ]:
debtrank_df.to_csv("DebtRank_Temporal_Results.csv", index=False)
files.download("DebtRank_Temporal_Results.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
import imageio
import os

os.makedirs("frames", exist_ok=True)

def get_node_color(distress):
    if distress >= 0.8:
        return "red"
    elif distress >= 0.5:
        return "orange"
    elif distress > 0:
        return "yellow"
    else:
        return "lightgreen"

frames = []

unique_times = sorted(debtrank_df['time'].unique())

for t in unique_times:

    distress_snapshot = debtrank_df[debtrank_df['time'] == t]


    loan_snapshot = loan_df[loan_df['time'] == t]

    G = nx.DiGraph()


    for _, row in distress_snapshot.iterrows():
        G.add_node(row['bank'], distress=row['debt_rank'])


    for _, row in loan_snapshot.iterrows():
        if row['source'] in G.nodes() and row['target'] in G.nodes():
             if not G.has_edge(row['source'], row['target']):
                G.add_edge(row['source'], row['target'], weight=row['exposure'])

    plt.figure(figsize=(10, 6))

    pos = nx.spring_layout(G, seed=42)
    node_colors = [get_node_color(G.nodes[n]['distress']) for n in G.nodes()]

    node_labels = {n: n for n in G.nodes()}

    nx.draw(G, pos, labels=node_labels, node_color=node_colors, edge_color='gray', node_size=500, font_size=8)
    plt.title(f"Distress Propagation at Time {t}")
    filename = f"frames/frame_{t}.png"
    plt.savefig(filename)
    plt.close()

    if os.path.exists(filename):
        frames.append(imageio.imread(filename))
    else:
        print(f"Warning: Frame file not created for time {t}")


<ipython-input-8-774991083>:61: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  frames.append(imageio.imread(filename))


In [ ]:

imageio.mimsave("distress_propagation.gif", frames, duration=1)

from google.colab import files
files.download("distress_propagation.gif")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>